In [41]:
with open('input.txt') as f:
    text = f.read()

In [42]:
with open('test.txt') as f:
    text_test = f.read()

## part 1

In [47]:
class MyDict:
    def __init__(self, src, dest, length) -> None:
        self.src = src
        self.dest = dest
        self.length = length
    def query_member(self, mem):
        if (mem >= self.src) & (mem < self.src + self.length):
            return self.dest + (mem - self.src)
        return mem
    
def map_str_to_range(s: str):
    dest, src, length = [int(i) for i in s.split()] 
    return {range(src, src+length):range(dest, dest+length)}

def map_range_to_dict(s: str):
    dest, src, length = [int(i) for i in s.split()] 
    return MyDict(src=src, dest=dest, length=length)

def get_from_MyDicts(dictionary_list, value):
    sub_result = list(set([i.query_member(value) for i in dictionary_list]).difference({value,}))
    if len(sub_result) == 1:
        return sub_result[0]
    return value

def map_all(seed, data_mappings):
    return get_from_MyDicts(
        data_mappings['humidity-to-location map'],
        get_from_MyDicts(
            data_mappings['temperature-to-humidity map'],
            get_from_MyDicts(
                data_mappings['light-to-temperature map'],
                get_from_MyDicts(
                    data_mappings['water-to-light map'],
                    get_from_MyDicts(
                        data_mappings['fertilizer-to-water map'],
                        get_from_MyDicts(
                            data_mappings['soil-to-fertilizer map'],
                            get_from_MyDicts(
                                    data_mappings['seed-to-soil map'],
                                    seed
                            )
                        )
                    )
                    )
            )
        )
    )

In [67]:
def solution_1(text):
    data = {i.split(':')[0]: i.split(':')[1].strip('\n').split('\n') for i in text.split('\n\n')}
    seeds = [int(i) for i in data['seeds'][0].split()]
    data_mappings = {}
    for key in data.keys():
        if key == 'seeds': continue
        data_mappings[key] = [map_range_to_dict(s) for s in data[key]]
    return min([map_all(s, data_mappings) for s in seeds])

solution_1(text)

178159714

## part 2

In [48]:
from collections import ChainMap
data = {i.split(':')[0]: i.split(':')[1].strip('\n').split('\n') for i in text_test.split('\n\n')}
seeds = [int(i) for i in data['seeds'][0].split()]
seed_ranges = []
for i in range(0, len(seeds), 2):
    seed_ranges.append(range(seeds[i], seeds[i]+seeds[i+1])) 
data_mappings = []
for key in data.keys():
    if key == 'seeds': continue
    data_mappings.append(
        dict(ChainMap(*sorted([map_str_to_range(s) for s in data[key]], key = lambda x: list(x.keys())[0].start, reverse=True)))
        )
data_mappings

[{range(50, 98): range(52, 100), range(98, 100): range(50, 52)},
 {range(0, 15): range(39, 54),
  range(15, 52): range(0, 37),
  range(52, 54): range(37, 39)},
 {range(0, 7): range(42, 49),
  range(7, 11): range(57, 61),
  range(11, 53): range(0, 42),
  range(53, 61): range(49, 57)},
 {range(18, 25): range(88, 95), range(25, 95): range(18, 88)},
 {range(45, 64): range(81, 100),
  range(64, 77): range(68, 81),
  range(77, 100): range(45, 68)},
 {range(0, 69): range(1, 70), range(69, 70): range(0, 1)},
 {range(56, 93): range(60, 97), range(93, 97): range(56, 60)}]

In [74]:
def split_range(src: range, maps: list[range]) -> list[range] :
    """
    Splits the source range into subranges according to the other ranges.

    The other ranges are expected to be non-overlapping and to be contained
    within the source range.

    Args:
        src: A range object.
        maps: A list of range objects.

    Returns:
        A list of range objects.
    
    Example: split_ranges(range(0, 10), [range(2,4), range(6,9)]) == [range(0,2), range(2,4), range(4, 6), range(6,9), range(9,10)]
    """

    result = []
    current_range = src
    for other_range in maps:
        if current_range.stop < other_range.start:
            result.append(current_range)
            current_range = None
            break
        elif current_range.start >= other_range.stop:
            pass
        elif current_range.start >= other_range.start:
            if current_range.stop >= other_range.stop:
                result.append(range(current_range.start, other_range.stop))
                current_range = range(other_range.stop, current_range.stop)
            else:
                result.append(range(current_range.start, current_range.stop))
                current_range = None
                break
        else:
            result.append(range(current_range.start, other_range.start))
            if current_range.stop < other_range.stop:
                result.append(range(other_range.start, current_range.stop))
                current_range = None
                break
            else:
                result.append(range(other_range.start, other_range.stop))
                current_range = range(other_range.stop, current_range.stop)
    if current_range: result.append(current_range)
    return result

def split_range_and_map(src: range, maps: dict) -> list[range] :
    """
    Splits the source range into subranges according to the other ranges.

    The other ranges are expected to be non-overlapping and to be contained
    within the source range.

    Args:
        src: A range object.
        maps: A list of range objects.

    Returns:
        A list of range objects.
    
    Example: split_ranges(range(0, 10), [range(2,4), range(6,9)]) == [range(0,2), range(2,4), range(4, 6), range(6,9), range(9,10)]
    """

    result = []
    current_range = src
    for other_range, other_dest in maps.items():
        if current_range.stop < other_range.start:
            result.append(current_range)
            current_range = None
            break
        elif current_range.start >= other_range.stop:
            pass
        elif current_range.start >= other_range.start:
            if current_range.stop >= other_range.stop:
                result.append(range(other_dest.start+current_range.start-other_range.start, other_dest.stop))
                current_range = range(other_range.stop, current_range.stop)
            else:
                result.append(range(other_dest.start+current_range.start-other_range.start, other_dest.start+current_range.stop-other_range.start))
                current_range = None
                break
        else:
            result.append(range(current_range.start, other_range.start))
            if current_range.stop < other_range.stop:
                result.append(range(other_dest.start, other_dest.start+current_range.stop-other_range.start))
                current_range = None
                break
            else:
                result.append(range(other_dest.start, other_dest.stop))
                current_range = range(other_range.stop, current_range.stop)
    if current_range: result.append(current_range)
    return result


source_range = range(7, 8)
other_ranges = [range(2, 4), range(6, 9)]
split_range(source_range, other_ranges)

[range(7, 8)]

In [96]:
from itertools import chain
def apply_to_seeds(ranges, maps):
    return list(chain(*[split_range_and_map(r, maps) for r in ranges]))
from functools import reduce
def select_min_from_ranges(ranges: list):
    return min([i.start for i in ranges])

from collections import ChainMap

def solution_2(text):
    data = {i.split(':')[0]: i.split(':')[1].strip('\n').split('\n') for i in text.split('\n\n')}
    seeds = [int(i) for i in data['seeds'][0].split()]
    seed_ranges = []
    for i in range(0, len(seeds), 2):
        seed_ranges.append(range(seeds[i], seeds[i]+seeds[i+1])) 
    data_mappings = []
    for key in data.keys():
        if key == 'seeds': continue
        data_mappings.append(
        dict(ChainMap(*sorted([map_str_to_range(s) for s in data[key]], key = lambda x: list(x.keys())[0].start, reverse=True)))
        )
    return select_min_from_ranges(
    reduce(apply_to_seeds, data_mappings, seed_ranges)
)

solution_2(text)

100165128